# Model Pipeline

## Pipeline Setup

In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
local_path = "data/sensor_data.csv"

base_uri = f"s3://{default_bucket}/airdata"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-us-east-1-790237383528/airdata/sensor_data.csv


In [4]:
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

mse_threshold = ParameterFloat(name="MseThreshold", default_value=0.5)

## Preprocessing Script

In [5]:
%%writefile code/preprocessing.py
import argparse
import os
import requests
import tempfile

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.15
TEST_SPLIT = 0.15
TARGET_PARAM = "pm25"

base_dir = "/opt/ml/processing"

df_location = pd.read_csv(
    f"{base_dir}/input/sensor_data.csv"
)

# Split training
df_param = df_location[df_location['parameter'] == TARGET_PARAM]  # Filter data for this parameter
train_data = df_param.iloc[:int(len(df_param) * TRAIN_SPLIT)]
train_data = train_data.reset_index(drop=True)

# Split validation
val_data = df_param.iloc[int(len(df_param) * TRAIN_SPLIT):int(len(df_param) * TRAIN_SPLIT) + int(len(df_param) * VAL_SPLIT)]
val_data = val_data.reset_index(drop=True)

# Split testing
test_data = df_param.iloc[int(len(df_param) * TRAIN_SPLIT) + int(len(df_param) * VAL_SPLIT):int(len(df_param) * TRAIN_SPLIT) + int(len(df_param) * VAL_SPLIT) + int(len(df_param) * TEST_SPLIT)]
test_data = test_data.reset_index(drop=True)

# Normalize the training dataset
scaler = StandardScaler()
train_data.loc[:, "value"] = scaler.fit_transform(train_data["value"].values.reshape(-1, 1))
val_data.loc[:, "value"] = scaler.transform(val_data["value"].values.reshape(-1, 1))
test_data.loc[:, "value"] = scaler.transform(test_data["value"].values.reshape(-1, 1))

print("Train Data Shape:", train_data.shape)
print("Validation Data Shape:", val_data.shape)
print("Test Data Shape:", test_data.shape)

pd.DataFrame(train_data).to_csv(f"{base_dir}/train/train.csv")
pd.DataFrame(val_data).to_csv(f"{base_dir}/validation/validation.csv")
pd.DataFrame(test_data).to_csv(f"{base_dir}/test/test.csv")

Overwriting code/preprocessing.py


In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor

framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-airdata-process",
    role=role,
    sagemaker_session=pipeline_session,
)

[02/17/25 17:08:25] INFO     Defaulting to only available Python version: py3                     ]8;id=441611;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=914147;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

In [7]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="code/preprocessing.py",
)

step_process = ProcessingStep(name="AirDataProcess", step_args=processor_args)

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Model Training

In [8]:
%%writefile code/train.py

import argparse
import os
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

# ---------------------------
# Dataset definition
# ---------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len=30, pred_len=1):
        """
        Args:
        - data (pd.DataFrame or np.array): Time series with a column 'value'
        - seq_len (int): Number of timesteps in the input sequence
        - pred_len (int): Number of timesteps in the output sequence
        """
        self.data = np.array(data["value"]) if isinstance(data, pd.DataFrame) else np.array(data)
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        """Returns total number of sequences available"""
        return max(0, len(self.data) - self.seq_len - self.pred_len)

    def __getitem__(self, idx):
        """Retrieves input sequence and target sequence"""
        if idx >= len(self):
            raise IndexError(f"Index {idx} out of bounds for dataset length {len(self)}")

        x = self.data[idx : idx + self.seq_len]
        y = self.data[idx + self.seq_len : idx + self.seq_len + self.pred_len]

        # Return tensors for PyTorch
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# ---------------------------
# Model definition
# ---------------------------
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x.unsqueeze(-1), (h0, c0))
        # out is (batch_size, seq_len, hidden_size)
        # We want the last timestep
        out = out[:, -1, :]
        out = self.fc(out)  # shape (batch_size, 1)
        return out

# ---------------------------
# Main training function
# ---------------------------
def main():
    parser = argparse.ArgumentParser()

    # Channels for data paths (SageMaker will populate these automatically)
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--validation", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))

    # Hyperparameters
    parser.add_argument("--seq-len", type=int, default=20)
    parser.add_argument("--pred-len", type=int, default=1)
    parser.add_argument("--batch-size", type=int, default=8)
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--hidden-size", type=int, default=50)
    parser.add_argument("--num-layers", type=int, default=1)
    parser.add_argument("--lr", type=float, default=0.001)

    args = parser.parse_args()

    # ---------------------------
    # Load data
    # ---------------------------
    # Processor wrote "train.csv", "validation.csv", and "test.csv"
    # into the respective directories: /opt/ml/input/data/train, etc.
    train_csv = os.path.join(args.train, "train.csv")
    val_csv   = os.path.join(args.validation, "validation.csv")
    test_csv  = os.path.join(args.test, "test.csv")

    train_data = pd.read_csv(train_csv)
    val_data   = pd.read_csv(val_csv)
    test_data  = pd.read_csv(test_csv)

    # ---------------------------
    # Create PyTorch datasets & loaders
    # ---------------------------
    train_dataset = TimeSeriesDataset(train_data, seq_len=args.seq_len, pred_len=args.pred_len)
    val_dataset   = TimeSeriesDataset(val_data, seq_len=args.seq_len, pred_len=args.pred_len)
    test_dataset  = TimeSeriesDataset(test_data, seq_len=args.seq_len, pred_len=args.pred_len)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    # ---------------------------
    # Initialize model, loss, optimizer
    # ---------------------------
    model = LSTM(
        input_size=1,
        hidden_size=args.hidden_size,
        num_layers=args.num_layers
    )

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # ---------------------------
    # Training Loop
    # ---------------------------
    for epoch in range(args.epochs):
        # Training
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_targets).item()
        val_loss /= len(val_loader)

        # Test
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for test_inputs, test_targets in test_loader:
                test_outputs = model(test_inputs)
                test_loss += criterion(test_outputs, test_targets).item()
        test_loss /= len(test_loader)

        print(f"Epoch [{epoch+1}/{args.epochs}] "
              f"TrainLoss: {train_loss:.4f} "
              f"ValLoss: {val_loss:.4f} "
              f"TestLoss: {test_loss:.4f}")
        
    # ---------------------------
    # Save the model
    # ---------------------------
    # Uploads /opt/ml/model to S3 after training
    model_dir = os.environ.get("SM_MODEL_DIR", "/opt/ml/model")
    model_path = os.path.join(model_dir, "model.pt")
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

if __name__ == "__main__":
    main()


Overwriting code/train.py


In [9]:
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

pytorch_estimator = PyTorch(
    entry_point="train.py",
    source_dir="code",
    role=role,
    framework_version="2.0",
    py_version="py310",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    hyperparameters={
        "seq-len": 20,
        "pred-len": 1,
        "batch-size": 8,
        "epochs": 50,
        "hidden-size": 50,
        "num-layers": 1,
        "lr": 0.001
    },
    sagemaker_session=pipeline_session,
    metric_definitions=[
        {
            "Name": "test_mse",
            "Regex": r"TestLoss:\s+([0-9\.]+)"
        }
    ]
)


In [10]:
step_train = TrainingStep(
    name="TrainStep",
    estimator=pytorch_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri
        ),
    },
)

## Inference Code

In [11]:
%%writefile code/inference.py

import json
import os
import torch
import numpy as np
from torch import nn

class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x.unsqueeze(-1), (h0, c0))
        # out is (batch_size, seq_len, hidden_size)
        # We want the last timestep
        out = out[:, -1, :]
        out = self.fc(out)  # shape (batch_size, 1)
        return out

def model_fn(model_dir):
    """
    Loads the model from the model_dir. This is invoked by SageMaker once at
    container startup to initialize your model. The returned object is passed
    to `predict_fn` for every inference request.
    """
    # Create model with the same architecture/hyperparams as training
    model = LSTM(input_size=1, hidden_size=50, num_layers=1)
    # Load state dict from model.pt
    model_path = os.path.join(model_dir, "model.pt")
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    model.eval()
    return model

def input_fn(request_body, request_content_type):
    """
    Deserializes the incoming request body into a PyTorch tensor.
    - If you expect JSON, parse it.
    - If you expect CSV, parse differently, etc.
    """
    if request_content_type == "application/json":
        # Example: request_body = '{"data": [12.3, 45.6, 78.9, ...]}'
        data = json.loads(request_body)["data"]
        # Convert to a float32 tensor. Suppose it's a 1D series (seq_len).
        inputs = torch.tensor([data], dtype=torch.float32)
        return inputs
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_object, model):
    """
    Performs prediction on the deserialized input.
    """
    with torch.no_grad():
        # input_object shape = (batch=1, seq_len)
        preds = model(input_object)  # shape = (batch=1, 1)
    return preds

def output_fn(prediction, response_content_type):
    """
    Serializes the prediction output.
    """
    if response_content_type == "application/json":
        # Convert the tensor to a Python float
        result = prediction.squeeze().item()  # single float
        return json.dumps({"prediction": result})
    else:
        raise ValueError(f"Unsupported response content type: {response_content_type}")


Overwriting code/inference.py


In [12]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.workflow.model_step import ModelStep

inference_model = PyTorchModel(
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    sagemaker_session=pipeline_session,
    framework_version="2.0",
    py_version="py310",
    entry_point="inference.py",
    source_dir="code",
)

step_create = ModelStep(
    name="AirDataCreateModel",
    step_args=inference_model.create(instance_type="ml.m5.large"),
)

In [13]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model_metrics = None  # metrics we want?
register_args = inference_model.register(
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name="AirDataModelGroup",
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)

step_register = ModelStep(
    name="RegisterAirDataModel",
    step_args=register_args
)

step_register = ModelStep(name="AirDataRegisterModel", step_args=register_args)

## Model Evaluation

In [ ]:
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join

step_fail = FailStep(
    name="MSEFail",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", mse_threshold])
)

test_mse = step_train.properties.FinalMetricDataList[0].Value

step_check_loss = ConditionStep(
    name="CheckTestLoss",
    conditions=[
        ConditionLessThanOrEqualTo(
            left=test_mse,
            right=mse_threshold
        )
    ],
    if_steps=[
        step_create, step_register
    ],
    else_steps=[step_fail],
)


## Create Pipeline

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"AirDataPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        mse_threshold,
    ],
    steps=[
        step_process,
        step_train,
        step_check_loss,
    ],
)

In [ ]:
import json

definition = json.loads(pipeline.definition())
definition

## Execute Pipeline

In [ ]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()
try:
    execution.wait()
except Exception as error:
    print(error)

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()

## Call Inference Endpoint
*I attempted to make endpoint deployment part of the pipeline but encountered a lot of issues with this. Keeping it manual.*

In [ ]:
predictor = inference_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge"
)

response = predictor.predict({"data": [12.3, 45.6, 78.9]})
print(response)